In [2]:
import pandas as pd
import gspread

In [3]:
sa = gspread.service_account(filename='geprek-wani-c700b4b8002f.json')

In [4]:
sheet = sa.open('Wani Spreadsheet')

In [5]:
worksheet = sheet.worksheet('Pesanan')

In [33]:
df = pd.DataFrame(worksheet.get_all_records())

In [34]:
df.rename(columns={'Request cabe makanan utama': 'Cabe'}, inplace=True)

In [35]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,06/04/2023 11:43:42,Rey,Nasi Ayam Geprek,,Tahu Crispy,Es teh,
1,06/04/2023 11:45:58,Abel,Nasi Ayam Geprek,1,Tahu Crispy,Es teh,Sambal Bawang
2,07/04/2023 11:45:58,Toto,Nasi Ayam Geprek,2,Kulit Crispy,,
3,08/04/2023 11:45:58,Timothy,Nasi Ayam Geprek,3,,,
4,09/04/2023 11:45:58,Averill,Nasi Ayam Geprek,2,,,


In [36]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1
df[list] = df[list].replace('', 1)

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Ayam Geprek,1,Rey
1,Nasi Ayam Geprek,1,Abel
2,Nasi Ayam Geprek,2,Toto
3,Nasi Ayam Geprek,3,Timothy
4,Nasi Ayam Geprek,2,Averill
5,Nasi Ayam Geprek,2,Toti
6,Nasi Ayam Geprek,3,MW
7,Nasi Ayam Geprek,3,Tota
8,Nasi Ayam Geprek,2,MC
9,Ayam Geprek tanpa nasi,1,Tata


In [37]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal
0,06/04/2023 11:43:42,Rey,Nasi Ayam Geprek,1,Tahu Crispy,Es teh,
1,06/04/2023 11:45:58,Abel,Nasi Ayam Geprek,1,Tahu Crispy,Es teh,Sambal Bawang
2,07/04/2023 11:45:58,Toto,Nasi Ayam Geprek,2,Kulit Crispy,,
3,08/04/2023 11:45:58,Timothy,Nasi Ayam Geprek,3,,,
4,09/04/2023 11:45:58,Averill,Nasi Ayam Geprek,2,,,
5,10/04/2023 11:45:58,Toti,Nasi Ayam Geprek,2,,,
6,11/04/2023 11:45:58,MW,Nasi Ayam Geprek,3,,,
7,12/04/2023 11:45:58,Tota,Nasi Ayam Geprek,3,,,
8,13/04/2023 11:45:58,MC,Nasi Ayam Geprek,2,,,
9,14/04/2023 11:45:58,Tata,Ayam Geprek tanpa nasi,1,,,


In [38]:
# Dataframe non-geprek
df_not_geprek = df[~df['Makanan Utama'].str.contains('Geprek')]

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa nasi')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

In [41]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-04-07 11:58:09.971160
2 Nasi Ayam Geprek Cabe 1 (Rey, Abel)
4 Nasi Ayam Geprek Cabe 2 (Toto, Averill, Toti, MC)
3 Nasi Ayam Geprek Cabe 3 (Timothy, MW, Tota)
1 Ayam Geprek tanpa nasi Cabe 1 (Tata)
2 Nasi Ayam Bakar (Hehe, Hoho)
2 Tahu Crispy (Rey, Abel)
1 Kulit Crispy (Toto)
2 Es teh (Rey, Abel)
1 Sambal Bawang (Abel)
